In [1]:
import pickle
import numpy as np
import pandas as pd
from sklearn.linear_model import LassoLarsCV, LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectFromModel
from scipy.stats import gmean
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from xgboost import DMatrix


from sklearn.ensemble import VotingClassifier
from mlxtend.classifier import StackingClassifier

In [2]:
#conda install -c anaconda py-xgboost

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
from sklearn.model_selection import KFold, cross_val_score, train_test_split


def acc(model):
    kf = KFold(n_splits=5, shuffle=True, random_state=42).get_n_splits(X_train.values)
    cv_auc = cross_val_score(model,X_train ,y_train, cv = kf, scoring = "roc_auc")
    return (cv_auc)


In [5]:
X_train = pd.read_csv('xtrain10.csv')
y_train = pd.read_csv('ytrain.csv')

In [6]:
y_train

,TARGET
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
76015,0.0
76016,0.0
76017,0.0
76018,0.0


In [7]:
X_train.drop('Unnamed: 0',axis=1, inplace = True)

In [8]:
y_train = np.ravel(y_train)

In [9]:
y_train

array([0., 0., 0., ..., 0., 0., 0.])

In [10]:
X_train

,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,n0
0,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39205.170000,321
1,2,34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49278.030000,295
2,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67333.770000,306
3,2,37,0.0,195.0,195.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64007.970000,275
4,2,39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117310.979016,285
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76015,2,48,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60926.490000,321
76016,2,39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,118634.520000,290
76017,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,74028.150000,307
76018,2,25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,84278.160000,306


In [11]:
type(y_train)

numpy.ndarray

# Log

In [12]:
# log model
logmodel = Pipeline([
        ("scl", StandardScaler()),
        ("clf", LogisticRegressionCV(penalty="l2", cv=5, max_iter=500, scoring="roc_auc", 
                                     n_jobs=-1 ,random_state=1))
    ])

logmodel.fit(X_train,y_train)


Pipeline(memory=None,
         steps=[('scl',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('clf',
                 LogisticRegressionCV(Cs=10, class_weight=None, cv=5,
                                      dual=False, fit_intercept=True,
                                      intercept_scaling=1.0, l1_ratios=None,
                                      max_iter=500, multi_class='auto',
                                      n_jobs=-1, penalty='l2', random_state=1,
                                      refit=True, scoring='roc_auc',
                                      solver='lbfgs', tol=0.0001, verbose=0))],
         verbose=False)

In [13]:
log_score = np.mean(acc(logmodel))

In [14]:
print(log_score)

0.7946184598894533


# XGB1

In [15]:
xgbc = xgb.XGBClassifier(max_depth=5, n_estimators=200, learning_rate=0.03, nthread=4, 
                          subsample=0.6815, colsample_bytree=0.701, seed=1234)
xgbc.fit(X_train, y_train, eval_metric="auc", verbose=2)

xgc1_score = np.mean(acc(xgbc))

In [16]:
print(xgc1_score)

0.8405885105988075


# XGB2

In [17]:
xgbc2 = xgb.XGBClassifier(max_depth=5, n_estimators=70, learning_rate=0.1, nthread=4, gamma=0.2,
                          subsample=0.4, colsample_bytree=0.7, min_child_weight=1, seed=1)

xgbc2.fit(X_train, y_train, eval_metric="auc", verbose=0)



sgbc2_score = np.mean(acc(xgbc2))

In [18]:
print(sgbc2_score)

0.8390156887793863


# RF

In [19]:
rfc = RandomForestClassifier(n_estimators=10, criterion="entropy", max_features=None, max_depth=7,
                             min_samples_leaf=9, n_jobs=4, random_state=1)
#rfc.fit(X_train, y_train)

rf_score = np.mean(acc(rfc))

In [20]:
print(rf_score)

0.8330861304593332


# AVG

In [21]:
from sklearn.ensemble import VotingClassifier
estimators = [('log', logmodel), ('xgb1', xgbc),  ('xgb2' , xgbc2), ('RF' , rfc)
              ]
av_model = VotingClassifier(estimators = estimators, weights =[0.1,0.3,0.3,0.3], voting='soft')

In [22]:
avg_score = np.mean(acc(av_model))

In [23]:
print(avg_score)

0.8399632779667936


# Stacking

In [24]:
reg_stack = StackingClassifier(classifiers=[logmodel,xgbc2, rfc],
                              meta_classifier=xgbc,use_features_in_secondary=True)
stk_score =np.mean(acc(reg_stack))


In [25]:
print(stk_score)

0.8403795631653725


In [26]:
X_test = pd.read_csv('xtest10.csv')
X_test.drop('Unnamed: 0',axis=1, inplace = True)

In [27]:
X_test

,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,n0
0,2,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40532.100000,304
1,2,35,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,45486.720000,307
2,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46993.950000,288
3,2,24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,187898.610000,321
4,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,73649.730000,306
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75813,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40243.200000,319
75814,2,26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,146961.300000,306
75815,2,24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,167299.770000,306
75816,2,40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117310.979016,306


In [28]:
X_train

,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,n0
0,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39205.170000,321
1,2,34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49278.030000,295
2,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67333.770000,306
3,2,37,0.0,195.0,195.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64007.970000,275
4,2,39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117310.979016,285
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76015,2,48,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60926.490000,321
76016,2,39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,118634.520000,290
76017,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,74028.150000,307
76018,2,25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,84278.160000,306


In [29]:
reg_stack.fit(X_train,y_train)

StackingClassifier(average_probas=False,
                   classifiers=[Pipeline(memory=None,
                                         steps=[('scl',
                                                 StandardScaler(copy=True,
                                                                with_mean=True,
                                                                with_std=True)),
                                                ('clf',
                                                 LogisticRegressionCV(Cs=10,
                                                                      class_weight=None,
                                                                      cv=5,
                                                                      dual=False,
                                                                      fit_intercept=True,
                                                                      intercept_scaling=1.0,
                                                  

In [30]:
y_pred= reg_stack.predict_proba(X_test)

In [31]:
y_pred = y_pred[:,1]

In [32]:
samplesubmission = pd.read_csv('sample_submission.csv')

output = pd.DataFrame({'ID': samplesubmission.ID, 'TARGET': y_pred})
output.to_csv('submission10.csv', index=False)